<a href="https://colab.research.google.com/github/arohikate241433/resume-analyser/blob/main/Agentic_Resume_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AGENTIC RESUME ANALYZER - FULLY AUTONOMOUS AI AGENT**

PROJECT: Applied Artificial Intelligence - Agentic AI System

AUTHOR: AROHI KATE

APPROACH: Framework-free implementation using pure Python + Hugging Face

**AGENTIC FEATURES DEMONSTRATED:**
1. ***Autonomous Decision Making*** - LLM-based tool selection (no hardcoded rules)
2. ***Goal Decomposition*** - Multi-step task planning
3. ***Self-Reflection*** - Agent evaluates outputs and self-corrects
4. ***State Management*** - Persistent memory across interactions
5. ***Structured Reasoning*** - JSON-based decisions with confidence scoring



In [ ]:
!pip install -q transformers torch accelerate PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from datetime import datetime
import json
import re

# Optional: PDF/TXT file support
try:
    import PyPDF2
    PDF_SUPPORT = True
except ImportError:
    PDF_SUPPORT = False
    print("⚠️ PyPDF2 not installed. PDF upload disabled. Install: pip install PyPDF2")

# ============================================================================
# AGENT STATE - Stateful Agent Implementation
# ============================================================================
"""
Why State Management?
- Agents need memory to track context across interactions
- Enables follow-up queries without repeating information
- Supports self-reflection by remembering past actions
- Demonstrates true autonomy (not just stateless request-response)
"""

agent_state = {
    "current_goal": "",              # What the agent is currently doing
    "resume_received": False,        # Has user provided resume?
    "last_tool_used": "",           # Track most recent action
    "ats_score": None,              # Calculated ATS score (0-100)
    "confidence_level": "",         # High/Medium/Low confidence
    "last_analysis": None,          # Store analysis results for reuse
    "task_plan": [],                # Multi-step execution plan
    "execution_history": []         # Log of all agent actions
}

def update_agent_state(key, value):
    """
    Centralized state update function with logging.

    Design Pattern: Single responsibility for state changes
    - All state updates go through this function
    - Provides consistent logging for debugging
    - Prevents scattered state modifications
    """
    agent_state[key] = value
    print(f"📊 Agent State Updated: {key} = {value}")

def log_execution(tool_name, result_summary):
    """
    Track agent's execution history for self-reflection.

    Why this matters:
    - Agent can review what it's already done
    - Prevents redundant actions (don't analyze same resume twice)
    - Enables meta-cognitive reasoning about past decisions
    """
    agent_state["execution_history"].append({
        "tool": tool_name,
        "timestamp": datetime.now().strftime('%H:%M:%S'),
        "result": result_summary
    })

# ============================================================================
# KNOWLEDGE BASE - Retrieval Component
# ============================================================================
"""
Retrieval-Augmented Generation (RAG) Approach:
- Instead of relying solely on LLM knowledge, we use structured data
- Maps job roles to required skills
- Enables deterministic skill matching (explainable results)
- Can be easily expanded with more roles and skills

Real-world usage: Companies maintain such databases for hiring requirements
"""

KNOWLEDGE_BASE = {
    "Data Scientist": [
        "python", "machine learning", "statistics", "sql",
        "pandas", "numpy", "scikit-learn", "tensorflow", "pytorch"
    ],
    "Software Engineer": [
        "python", "java", "javascript", "git", "api",
        "backend", "frontend", "database", "testing"
    ],
    "ML Engineer": [
        "python", "machine learning", "deep learning",
        "tensorflow", "pytorch", "docker", "kubernetes", "mlops"
    ],
    "Full Stack Developer": [
        "javascript", "react", "node.js", "python",
        "sql", "html", "css", "git", "rest api"
    ],
    "Data Analyst": [
        "sql", "excel", "python", "data visualization",
        "tableau", "power bi", "statistics"
    ],
    "DevOps Engineer": [
        "linux", "docker", "kubernetes", "ci/cd",
        "aws", "azure", "terraform", "ansible"
    ],
    "Backend Developer": [
        "python", "java", "node.js", "sql", "api",
        "microservices", "redis", "mongodb"
    ],
}

# ============================================================================
# MEMORY - Short-term Context Storage
# ============================================================================
"""
Memory Component:
- Stores conversation history for context-aware responses
- Retains current resume to avoid re-uploading
- Enables natural follow-up queries like "extract skills from it"

Design Choice: Simple in-memory storage (no database needed for demo)
Production version would use persistent storage (SQLite, PostgreSQL)
"""

class AgentMemory:
    def __init__(self):
        self.conversation_history = []  # List of {role, content} dicts
        self.current_resume = None      # Currently loaded resume text

    def store_message(self, role, content):
        """Store user/assistant messages for conversation context"""
        self.conversation_history.append({"role": role, "content": content})

    def store_resume(self, resume_text):
        """
        Store resume in memory and update agent state.

        Key feature: Agent now "knows" resume is available
        - Enables follow-up queries without re-pasting
        - Triggers state update for tracking
        """
        self.current_resume = resume_text
        update_agent_state("resume_received", True)

    def get_resume(self):
        """Retrieve currently stored resume"""
        return self.current_resume

    def get_recent_history(self, n=3):
        """Get last N conversation turns for context"""
        return self.conversation_history[-n:]

# ============================================================================
# FILE HANDLING - PDF and TXT Support
# ============================================================================
"""
File Upload Feature:
- Allows users to upload resume files (.txt, .pdf)
- More professional than copy-pasting text
- Handles real-world resume formats

Note: PDF parsing can be imperfect (formatting issues)
"""

def read_txt_file(filepath):
    """
    Read plain text file.

    Args:
        filepath: Path to .txt file
    Returns:
        String containing file contents
    """
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"❌ Error reading TXT file: {str(e)}"

def read_pdf_file(filepath):
    """
    Extract text from PDF file.

    Args:
        filepath: Path to .pdf file
    Returns:
        String containing extracted text

    Note: Requires PyPDF2 library
    Complex PDFs (images, tables) may not parse perfectly
    """
    if not PDF_SUPPORT:
        return "❌ PyPDF2 not installed. Install with: pip install PyPDF2"

    try:
        with open(filepath, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ""

            # Extract text from each page
            for page_num, page in enumerate(pdf_reader.pages):
                text += page.extract_text()
                text += f"\n--- Page {page_num + 1} ---\n"

            return text
    except Exception as e:
        return f"❌ Error reading PDF file: {str(e)}"

def load_resume_file(filepath):
    """
    Universal file loader - handles both .txt and .pdf

    Usage: load_resume_file("resume.pdf")

    Returns:
        (success: bool, content: str)
    """
    if filepath.endswith('.txt'):
        content = read_txt_file(filepath)
        return (True, content) if "Error" not in content else (False, content)

    elif filepath.endswith('.pdf'):
        content = read_pdf_file(filepath)
        return (True, content) if "Error" not in content else (False, content)

    else:
        return (False, "❌ Unsupported format. Use .txt or .pdf files only.")

# ============================================================================
# LLM INITIALIZATION
# ============================================================================
"""
Model Selection: Google Gemma-2B-IT (Instruction-Tuned)

Why this model?
- Small enough for Colab free tier (2B parameters)
- Instruction-tuned for following prompts
- Good balance of quality and speed
- No API keys needed (runs locally)

Alternative models: Llama-3-8B, Mistral-7B (need more resources)
"""

print("🔧 Loading LLM model (this may take 30-60 seconds)...")
MODEL_NAME = "microsoft/phi-2"

# Load tokenizer (converts text to tokens)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Load model (the actual AI brain)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,  # Use float32 for CPU compatibility
    device_map="auto"            # Automatically use GPU if available
)

def generate_llm_response(prompt, max_length=300):
    """Generate response from LLM with correct device handling"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )

    # MOVE INPUTS TO MODEL DEVICE
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    prompt_text = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
    response = response[len(prompt_text):].strip()

    return response

# ============================================================================
# DECORATOR - Validation Helper
# ============================================================================
"""
Decorator Pattern for Resume Validation:

Problem: Every tool needs to check if resume exists
Old approach: if not memory.get_resume(): return error (repeated 6 times!)
New approach: @requires_resume decorator (DRY principle)

Benefits:
- Cleaner code (no repetition)
- Centralized validation logic
- Easy to modify validation rules
"""

def requires_resume(func):
    """
    Decorator that validates resume exists before tool execution.

    Usage:
        @requires_resume
        def tool_resume_analysis(memory):
            # This only runs if resume exists

    Python decorator magic: Wraps function with validation check
    """
    def wrapper(memory, *args, **kwargs):
        if not memory.get_resume():
            return {
                "success": False,
                "message": "⚠️ Please provide your resume text first."
            }
        return func(memory, *args, **kwargs)
    return wrapper

# ============================================================================
# AGENT CORE - Autonomous Decision Making
# ============================================================================
"""
CRITICAL AGENTIC COMPONENT: Decision Making

This is where "agentic" behavior happens:
- LLM decides which tool to use (not hardcoded if/else)
- Returns structured JSON with reasoning
- Includes confidence score

Traditional approach: if "score" in input → run scoring tool
Agentic approach: LLM reasons about intent → decides best action

Why JSON output?
- Structured and parseable
- Includes reasoning (explainability)
- Confidence scoring (agent knows when uncertain)
"""

def agent_decide_intent(user_input, memory):
    """
    Agent uses LLM to autonomously decide which tool to invoke.

    This is the CORE of agentic behavior - no hardcoded rules!

    Input: User's natural language query
    Output: {
        "tool": "TOOL_NAME",
        "reasoning": "Why this tool was chosen",
        "confidence": 0.95,  # How sure the agent is
        "multi_step": false
    }

    Design Pattern: Prompt engineering for structured outputs
    - We instruct LLM to return ONLY JSON
    - Provide clear examples of available tools
    - Include context (resume status, recent actions)
    """

    # Build context for better decision-making
    resume_context = "Resume is stored in memory." if memory.get_resume() else "No resume in memory."

    # Get recent execution history to avoid redundant actions
    recent_tools = agent_state.get("execution_history", [])[-3:]
    recent_context = f"Recently executed: {[t['tool'] for t in recent_tools]}" if recent_tools else "No recent executions"

    # Construct decision prompt (this is prompt engineering!)
    decision_prompt = f"""You are an AI agent that decides what action to take. Respond ONLY with valid JSON.

Available Tools:
- RESUME_ANALYSIS: Match resume to job roles
- SKILL_EXTRACTION: Extract technical skills
- RESUME_IMPROVEMENT: Suggest improvements
- ATS_SCORING: Calculate ATS compatibility score
- GENERATE_REPORT: Create evaluation report
- FULL_EVALUATION: Complete multi-step analysis (runs all tools autonomously)
- GENERAL_CHAT: Handle casual questions

Context: {resume_context}
{recent_context}

User Input: {user_input}

Decision Rules:
- If user wants "full/complete/comprehensive evaluation", choose FULL_EVALUATION
- If user provides resume text (>50 words), choose RESUME_ANALYSIS
- For specific tasks (scoring, skills, etc.), choose appropriate single tool
- For casual chat unrelated to resumes, choose GENERAL_CHAT

Respond with JSON only (no other text):
{{"tool": "TOOL_NAME", "reasoning": "why this tool", "confidence": 0.95, "multi_step": false}}

JSON:"""

    # Generate LLM decision
    response = generate_llm_response(decision_prompt, max_length=100)

    # Parse JSON response with robust error handling
    try:
        # Try to extract JSON from response (handles markdown code blocks)
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            decision_data = json.loads(json_match.group())
        else:
            decision_data = json.loads(response)

        # Validate required fields exist
        if "tool" not in decision_data:
            raise ValueError("Missing 'tool' field in JSON")

        # Normalize tool name (uppercase, replace spaces with underscores)
        tool = decision_data["tool"].upper().replace(" ", "_")

        # Validate tool exists in our system
        valid_tools = [
            "RESUME_ANALYSIS", "SKILL_EXTRACTION", "RESUME_IMPROVEMENT",
            "ATS_SCORING", "GENERATE_REPORT", "FULL_EVALUATION", "GENERAL_CHAT"
        ]

        # Fuzzy matching if exact match not found
        if tool not in valid_tools:
            for valid_tool in valid_tools:
                if valid_tool in tool or tool in valid_tool:
                    tool = valid_tool
                    break
            else:
                tool = "GENERAL_CHAT"  # Safe fallback

        # Ensure all fields exist with defaults
        decision_data["tool"] = tool
        decision_data.setdefault("confidence", 0.7)
        decision_data.setdefault("reasoning", "Decision based on user input")
        decision_data.setdefault("multi_step", False)

        return decision_data

    except (json.JSONDecodeError, ValueError) as e:
        """
        Fallback mechanism if JSON parsing fails.

        Why needed? LLMs sometimes ignore instructions and return plain text
        Solution: Keyword-based fallback (last resort, not ideal)
        """
        response_upper = response.upper()

        # Simple keyword matching as fallback
        if "FULL" in response_upper or "COMPLETE" in response_upper:
            tool = "FULL_EVALUATION"
        elif "RESUME" in response_upper and "ANALYS" in response_upper:
            tool = "RESUME_ANALYSIS"
        elif "SKILL" in response_upper:
            tool = "SKILL_EXTRACTION"
        elif "IMPROVE" in response_upper:
            tool = "RESUME_IMPROVEMENT"
        elif "ATS" in response_upper or "SCORE" in response_upper:
            tool = "ATS_SCORING"
        elif "REPORT" in response_upper:
            tool = "GENERATE_REPORT"
        else:
            tool = "GENERAL_CHAT"

        return {
            "tool": tool,
            "reasoning": "Fallback decision based on keywords",
            "confidence": 0.6,
            "multi_step": False
        }

# ============================================================================
# GOAL DECOMPOSITION - Multi-Step Planning
# ============================================================================
"""
AGENTIC FEATURE: Goal Decomposition

What is this?
- Agent breaks complex goals into sequential steps
- Plans entire workflow before execution
- Demonstrates autonomous planning ability

Example:
User: "Fully evaluate my resume"
Agent thinks: This needs analysis → scoring → improvements → report
Agent creates plan: [STEP1, STEP2, STEP3, STEP4]
Agent executes all steps without asking!

This is what separates reactive bots from intelligent agents.
"""

def create_task_plan(goal, memory):
    """
    Agent creates multi-step execution plan for complex goals.

    Planning Logic:
    1. Check execution history (don't repeat done work)
    2. Determine required steps based on goal
    3. Order steps logically (analysis before scoring)
    4. Return sequential plan

    Args:
        goal: High-level goal (e.g., "FULL_EVALUATION")
        memory: Agent memory for context

    Returns:
        List of tool names to execute in order
    """

    if goal == "FULL_EVALUATION":
        # Check what's already been done (avoid redundancy)
        history = [h["tool"] for h in agent_state.get("execution_history", [])]

        plan = []

        # Step 1: Analyze resume (foundational step)
        if "RESUME_ANALYSIS" not in history:
            plan.append("RESUME_ANALYSIS")

        # Step 2: Extract skills (detailed breakdown)
        if "SKILL_EXTRACTION" not in history:
            plan.append("SKILL_EXTRACTION")

        # Step 3: Calculate ATS score (quantitative assessment)
        if "ATS_SCORING" not in history:
            plan.append("ATS_SCORING")

        # Step 4: Generate improvements (actionable recommendations)
        # Always include improvements (user wants fresh suggestions)
        plan.append("RESUME_IMPROVEMENT")

        # Step 5: Create comprehensive report (final deliverable)
        plan.append("GENERATE_REPORT")

        return plan

    # Single-step task (no decomposition needed)
    return [goal]

# ============================================================================
# TOOLS - Specialized Functions
# ============================================================================
"""
Tool-Based Architecture:

Why separate tools?
- Modularity: Each tool has single responsibility
- Reusability: Tools can be called by different workflows
- Testability: Easy to test individual tools
- Extensibility: Add new tools without changing core logic

Each tool returns: {"success": bool, "message": str, "data": dict}
Consistent interface makes composition easier
"""

@requires_resume  # Decorator ensures resume exists
def tool_resume_analysis(memory):
    """
    TOOL 1: Resume Analysis

    What it does:
    1. Extracts skills from resume text
    2. Matches skills against knowledge base (RAG)
    3. Calculates match percentage for each role
    4. Ranks roles by compatibility

    Technique: Retrieval-Augmented Generation (RAG)
    - Uses structured knowledge base
    - Deterministic matching (explainable)
    - No hallucination (unlike pure LLM)
    """
    resume_text = memory.get_resume()
    resume_lower = resume_text.lower()  # Case-insensitive matching

    # Extract skills by checking if they appear in resume
    detected_skills = []
    all_skills = set()

    # Collect all possible skills from knowledge base
    for skills in KNOWLEDGE_BASE.values():
        all_skills.update(skills)

    # Check which skills appear in resume
    for skill in all_skills:
        if skill in resume_lower:
            detected_skills.append(skill)

    # Match resume to job roles
    role_matches = {}
    for role, required_skills in KNOWLEDGE_BASE.items():
        # Find skills that match this role
        matching_skills = [s for s in required_skills if s in detected_skills]

        # Calculate match percentage
        match_percentage = (len(matching_skills) / len(required_skills)) * 100

        # Store results
        role_matches[role] = {
            "match": match_percentage,
            "found": matching_skills,
            "missing": [s for s in required_skills if s not in detected_skills]
        }

    # Sort roles by match percentage (best matches first)
    sorted_roles = sorted(role_matches.items(), key=lambda x: x[1]["match"], reverse=True)

    # Store results in agent state for later use
    agent_state["last_analysis"] = {
        "skills": detected_skills,
        "roles": sorted_roles
    }

    # Format output for user
    result = "📊 Resume Analysis Results:\n\n"
    result += f"✅ Detected Skills: {', '.join(detected_skills[:10])}\n\n"
    result += "🎯 Best Matching Roles:\n"

    # Show top 3 matching roles
    for role, data in sorted_roles[:3]:
        result += f"\n{role}: {data['match']:.0f}% match\n"
        result += f"  Found: {', '.join(data['found'][:5])}\n"
        if data['missing'][:3]:
            result += f"  Missing: {', '.join(data['missing'][:3])}\n"

    # Log execution for history tracking
    log_execution("RESUME_ANALYSIS", f"{len(detected_skills)} skills detected")

    return {
        "success": True,
        "message": result,
        "data": {"skills_count": len(detected_skills)}
    }

@requires_resume
def tool_skill_extraction(memory):
    """
    TOOL 2: Skill Extraction

    Extracts and categorizes technical skills.

    Categorization logic:
    - Programming: Languages (Python, Java, etc.)
    - Data/ML: ML-related skills
    - Tools/Platforms: Infrastructure and tools

    Use case: Detailed skill inventory for resume optimization
    """
    resume_text = memory.get_resume()
    resume_lower = resume_text.lower()

    # Get all skills from knowledge base
    all_skills = set()
    for skills in KNOWLEDGE_BASE.values():
        all_skills.update(skills)

    # Detect which skills appear in resume
    detected_skills = [skill for skill in all_skills if skill in resume_lower]

    # Categorize skills for better organization
    categorized = {
        "Programming": [],
        "Data/ML": [],
        "Tools/Platforms": []
    }

    # Simple categorization logic (can be enhanced with ML)
    for skill in detected_skills:
        if skill in ["python", "java", "javascript", "node.js"]:
            categorized["Programming"].append(skill)
        elif skill in ["machine learning", "deep learning", "statistics", "data visualization"]:
            categorized["Data/ML"].append(skill)
        else:
            categorized["Tools/Platforms"].append(skill)

    # Format output
    result = "🔍 Extracted Skills:\n\n"
    for category, skills in categorized.items():
        if skills:
            result += f"{category}: {', '.join(skills)}\n"

    log_execution("SKILL_EXTRACTION", f"{len(detected_skills)} skills extracted")

    return {
        "success": True,
        "message": result,
        "data": {"total_skills": len(detected_skills)}
    }

@requires_resume
def tool_resume_improvement(memory):
    """
    TOOL 3: Resume Improvement

    Uses LLM to generate actionable suggestions.

    Why use LLM here?
    - Improvements need creativity and context understanding
    - LLM can provide personalized recommendations
    - Complements rule-based scoring with human-like advice

    Prompt engineering: We give context (ATS score if available)
    """
    resume_text = memory.get_resume()

    # Include ATS score in context if available
    ats_score = agent_state.get("ats_score")
    score_context = f"Current ATS Score: {ats_score}/100. " if ats_score else ""

    # Construct improvement prompt
    improvement_prompt = f"""You are an expert resume advisor. {score_context}Analyze this resume and provide 3 specific, actionable improvements to make it more ATS-friendly and professional.

Resume:
{resume_text[:500]}

Provide exactly 3 improvements in this format:
1. [Specific improvement with example]
2. [Specific improvement with example]
3. [Specific improvement with example]

Improvements:"""

    # Generate suggestions using LLM
    suggestions = generate_llm_response(improvement_prompt, max_length=200)

    result = "💡 Resume Improvement Suggestions:\n\n"
    result += suggestions

    log_execution("RESUME_IMPROVEMENT", "Generated 3 suggestions")

    return {"success": True, "message": result}

@requires_resume
def tool_ats_scoring(memory):
    """
    TOOL 4: ATS Scoring (EXPLAINABLE AI)

    Calculates ATS compatibility score (0-100) with detailed explanation.

    Scoring Formula (weighted):
    - 70%: Skill match percentage
    - 20%: Keyword density
    - 10%: Resume length appropriateness

    Why these weights?
    - Skills are most important for ATS systems
    - Keywords help with search/ranking
    - Length matters (too short/long = red flag)

    Explainability: Every score includes breakdown and reasoning
    """
    resume_text = memory.get_resume()
    resume_lower = resume_text.lower()

    # Extract detected skills
    all_skills = set()
    for skills in KNOWLEDGE_BASE.values():
        all_skills.update(skills)

    detected_skills = [skill for skill in all_skills if skill in resume_lower]

    # Find best matching role (for targeted scoring)
    best_role = None
    best_match_percentage = 0
    best_required_skills = []

    for role, required_skills in KNOWLEDGE_BASE.items():
        matching_skills = [s for s in required_skills if s in detected_skills]
        match_percentage = (len(matching_skills) / len(required_skills)) * 100

        if match_percentage > best_match_percentage:
            best_match_percentage = match_percentage
            best_role = role
            best_required_skills = required_skills

    # SCORING COMPONENT 1: Skill Match (70% weight)
    skill_score = best_match_percentage * 0.7

    # SCORING COMPONENT 2: Keyword Density (20% weight)
    # More keywords = better ATS optimization
    keyword_count = sum(1 for skill in all_skills if skill in resume_lower)
    keyword_density = min(keyword_count / 15 * 100, 100) * 0.2

    # SCORING COMPONENT 3: Resume Length (10% weight)
    # Ideal: 300-1000 words (industry standard)
    word_count = len(resume_text.split())

    if 300 <= word_count <= 1000:
        length_score = 10  # Perfect length
    elif word_count < 300:
        length_score = (word_count / 300) * 10  # Too short penalty
    else:
        length_score = max(10 - (word_count - 1000) / 500, 0)  # Too long penalty

    # FINAL ATS SCORE (0-100)
    ats_score = int(skill_score + keyword_density + length_score)

    # CONFIDENCE LEVEL (meta-cognitive awareness)
    if ats_score >= 75:
        confidence = "High"
    elif ats_score >= 50:
        confidence = "Medium"
    else:
        confidence = "Low"

    # Update agent state
    update_agent_state("ats_score", ats_score)
    update_agent_state("confidence_level", confidence)

    # Generate explanation (EXPLAINABILITY)
    matched_skills = len([s for s in best_required_skills if s in detected_skills])
    total_skills = len(best_required_skills)

    # Detailed, human-readable explanation
    result = f"""🎯 ATS Compatibility Score: {ats_score}/100

📊 Score Breakdown:
• Best Matching Role: {best_role}
• Skills Matched: {matched_skills}/{total_skills} required skills
• Keyword Coverage: {int(keyword_density/0.2)}%
• Resume Length: {word_count} words

💡 Explanation:
Score is {ats_score}% because {matched_skills} out of {total_skills} required skills for {best_role} were detected.

🔍 Confidence Level: {confidence}
"""

    # Add recommendation if score is low
    if ats_score < 70:
        result += "\n⚠️ Recommendation: Add more relevant skills and optimize keywords for better ATS performance.\n"

    log_execution("ATS_SCORING", f"Score: {ats_score}/100")

    return {
        "success": True,
        "message": result,
        "data": {"score": ats_score, "confidence": confidence}
    }

@requires_resume
def tool_generate_report(memory):
    """
    TOOL 5: Automated Report Generation (AUTOMATION FEATURE)

    Automatically generates comprehensive evaluation report.

    Key automation aspects:
    - No user input needed (runs autonomously)
    - Gathers data from all previous analyses
    - Saves to file automatically
    - Includes complete execution history

    This demonstrates: Agent can perform complex tasks end-to-end
    """

    resume_text = memory.get_resume()
    analysis = agent_state.get("last_analysis")
    ats_score = agent_state.get("ats_score")
    confidence = agent_state.get("confidence_level")

    # Ensure we have required data (run analyses if missing)
    if not analysis:
        tool_resume_analysis(memory)
        analysis = agent_state.get("last_analysis")

    if not ats_score:
        tool_ats_scoring(memory)
        ats_score = agent_state.get("ats_score")
        confidence = agent_state.get("confidence_level")

    # Generate improvement suggestions using LLM
    improvement_prompt = f"""Provide 3 brief resume improvements for ATS optimization.

Resume snippet:
{resume_text[:300]}

List 3 improvements:"""

    improvements = generate_llm_response(improvement_prompt, max_length=150)

    # BUILD COMPREHENSIVE REPORT
    report_content = f"""
{'='*70}
AUTOMATED RESUME EVALUATION REPORT
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*70}

DETECTED SKILLS:
{', '.join(analysis['skills'][:15]) if analysis else 'N/A'}

{'='*70}
MATCHED ROLES:
"""

    if analysis and analysis['roles']:
        for role, data in analysis['roles'][:3]:
            report_content += f"\n{role}: {data['match']:.0f}% match\n"
            report_content += f"  Found Skills: {', '.join(data['found'][:5])}\n"

    report_content += f"""
{'='*70}
ATS COMPATIBILITY SCORE:
Score: {ats_score}/100
Confidence Level: {confidence}

EXPLANATION:
The ATS score reflects how well the resume aligns with industry standards
and applicant tracking system requirements based on skill matching, keyword
density, and resume structure optimization.

{'='*70}
IMPROVEMENT SUGGESTIONS:

{improvements}

{'='*70}
AGENT EXECUTION HISTORY:
"""

    # Include agent's execution trail (transparency)
    for execution in agent_state.get("execution_history", []):
        report_content += f"\n[{execution['timestamp']}] {execution['tool']}: {execution['result']}"

    report_content += f"""

{'='*70}
END OF REPORT
{'='*70}
"""

    # Save report to file with timestamp
    filename = f"resume_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(report_content)

        print(f"✅ Automated Report Generated: {filename}")
        log_execution("GENERATE_REPORT", f"Saved to {filename}")

        return {
            "success": True,
            "message": f"📄 Automated Resume Report Generated!\n\nReport saved to: {filename}\n\n{report_content[:400]}...\n\n(Full report saved to file)"
        }

    except Exception as e:
        return {
            "success": False,
            "message": f"❌ Error generating report: {str(e)}\n\nReport Content:\n{report_content[:400]}..."
        }

def tool_general_chat(user_input):
    """
    TOOL 6: General Chat

    Handles non-resume queries using LLM.

    Examples: "What is ATS?", "How are you?", "Tell me a joke"

    Design: Falls back to this when user isn't asking about resumes
    """
    chat_prompt = f"""You are a helpful AI assistant. Respond naturally to the user's question.

User: {user_input}
Assistant:"""

    response = generate_llm_response(chat_prompt, max_length=150)
    log_execution("GENERAL_CHAT", "Casual conversation")

    return {"success": True, "message": response}

# ============================================================================
# SELF-REFLECTION - Meta-Cognitive Reasoning
# ============================================================================
"""
AGENTIC FEATURE: Self-Reflection

What is this?
- Agent evaluates its own outputs
- Identifies when additional actions are needed
- Autonomously triggers corrective measures

Example:
Agent calculates ATS score = 62%
Agent reflects: "This is below 70%, user needs help"
Agent autonomously: Runs improvement tool
Result: User gets help without asking!

This demonstrates meta-cognition (thinking about thinking).
"""

def agent_self_reflect(memory):
    """
    Agent reflects on execution results and autonomously adapts.

    Reflection patterns:
    1. Low ATS score → Trigger improvements
    2. Analysis done but no score → Calculate score
    3. Recent improvements → Suggest updating report

    Returns:
        List of reflections with observations and suggested actions
    """

    ats_score = agent_state.get("ats_score")
    last_tool = agent_state.get("last_tool_used")

    reflections = []

    # REFLECTION 1: Low score triggers autonomous improvement
    if ats_score and ats_score < 70 and last_tool == "ATS_SCORING":
        reflections.append({
            "observation": f"ATS score is {ats_score}/100 (below 70% threshold)",
            "action": "RESUME_IMPROVEMENT",
            "reasoning": "Low score requires improvement suggestions"
        })

    # REFLECTION 2: Analysis without scoring is incomplete
    if last_tool == "RESUME_ANALYSIS" and not ats_score:
        reflections.append({
            "observation": "Resume analyzed but no ATS score calculated",
            "action": "ATS_SCORING",
            "reasoning": "Complete evaluation requires quantitative scoring"
        })

    # REFLECTION 3: After improvements, suggest report refresh
    if last_tool == "RESUME_IMPROVEMENT" and ats_score:
        reflections.append({
            "observation": "Improvements suggested after scoring",
            "action": None,  # Don't auto-execute, just inform
            "reasoning": "User may want updated report with new suggestions"
        })

    return reflections

# ============================================================================
# AGENT EXECUTOR - Main Orchestration Logic
# ============================================================================
"""
Agent Execution Flow:
1. Decide intent (which tool?)
2. Check if multi-step plan needed
3. Execute tool(s)
4. Self-reflect on results
5. Autonomously trigger follow-up actions if needed
6. Update state and memory

This is the "brain" that coordinates everything.
"""

def execute_single_tool(tool_name, memory, user_input=None):
    """
    Execute a single tool and return standardized result.

    Centralized execution point for consistency.

    Args:
        tool_name: Name of tool to execute
        memory: Agent memory object
        user_input: Original user input (for context)

    Returns:
        {"success": bool, "message": str, "data": dict}
    """

    if tool_name == "RESUME_ANALYSIS":
        # Check if user provided new resume in input
        if len(user_input or "") > 50:
            memory.store_resume(user_input)
        return tool_resume_analysis(memory)

    elif tool_name == "SKILL_EXTRACTION":
        return tool_skill_extraction(memory)

    elif tool_name == "RESUME_IMPROVEMENT":
        return tool_resume_improvement(memory)

    elif tool_name == "ATS_SCORING":
        return tool_ats_scoring(memory)

    elif tool_name == "GENERATE_REPORT":
        return tool_generate_report(memory)

    elif tool_name == "GENERAL_CHAT":
        return tool_general_chat(user_input or "")

    else:
        return {"success": False, "message": "⚠️ Unknown tool"}

def agent_execute(user_input, memory):
    """
    MAIN AGENT EXECUTION FUNCTION

    This orchestrates the entire agentic workflow:
    - Autonomous decision making
    - Multi-step planning and execution
    - Self-reflection and adaptation
    - State management

    Input: User's natural language query
    Output: Comprehensive response (possibly from multiple tools)
    """

    # STEP 1: AUTONOMOUS DECISION MAKING
    # Agent uses LLM to decide what to do (structured JSON output)
    decision_data = agent_decide_intent(user_input, memory)

    tool = decision_data["tool"]
    confidence = decision_data["confidence"]
    reasoning = decision_data["reasoning"]

    # Log decision with transparency
    print(f"\n🤖 Agent Decision: {tool}")
    print(f"💭 Reasoning: {reasoning}")
    print(f"📈 Confidence: {confidence:.0%}")

    update_agent_state("current_goal", tool)

    # STEP 2: MULTI-STEP EXECUTION (Goal Decomposition)
    if tool == "FULL_EVALUATION":
        """
        AUTONOMOUS MULTI-STEP PLANNING

        User says: "Fully evaluate my resume"
        Agent thinks: This needs 5 steps
        Agent plans: [analyze → extract → score → improve → report]
        Agent executes: All 5 steps without asking!

        This demonstrates true autonomy.
        """

        if not memory.get_resume():
            return "⚠️ Please provide your resume first for full evaluation."

        print("\n🧠 Agent Planning: Creating multi-step execution plan...")
        task_plan = create_task_plan(tool, memory)

        print(f"📋 Execution Plan: {' → '.join(task_plan)}")
        update_agent_state("task_plan", task_plan)

        results = []

        # Execute each step in the plan
        for step_num, step_tool in enumerate(task_plan, 1):
            print(f"\n{'='*70}")
            print(f"🔧 Executing Step {step_num}/{len(task_plan)}: {step_tool}")
            print(f"{'='*70}")

            result = execute_single_tool(step_tool, memory)
            results.append(result["message"])

            update_agent_state("last_tool_used", step_tool)

            # Self-reflect after each step
            reflections = agent_self_reflect(memory)
            if reflections:
                print(f"\n💭 Agent Self-Reflection:")
                for reflection in reflections:
                    print(f"   Observation: {reflection['observation']}")
                    if reflection['action']:
                        print(f"   Suggested Action: {reflection['action']}")

        # Combine all results into final response
        final_result = "\n\n".join(results)
        final_result = f"✅ FULL EVALUATION COMPLETED ({len(task_plan)} steps executed)\n\n{final_result}"

        return final_result

    # STEP 3: SINGLE TOOL EXECUTION
    else:
        print(f"\n🔧 Tool Executed: {tool}")

        result = execute_single_tool(tool, memory, user_input)
        update_agent_state("last_tool_used", tool)

        # STEP 4: SELF-REFLECTION (autonomous adaptation)
        reflections = agent_self_reflect(memory)

        if reflections:
            print(f"\n💭 Agent Self-Reflection:")
            for reflection in reflections:
                print(f"   Observation: {reflection['observation']}")

                # If reflection suggests action, execute it autonomously
                if reflection['action']:
                    print(f"   🤖 Autonomously executing: {reflection['action']}...")

                    # AUTONOMOUS ACTION EXECUTION
                    auto_result = execute_single_tool(reflection['action'], memory)
                    update_agent_state("last_tool_used", reflection['action'])

                    # Append autonomous action result to main result
                    result["message"] += f"\n\n{'='*70}\n🤖 AUTONOMOUS ACTION TRIGGERED\n{'='*70}\n\n{auto_result['message']}"

        return result["message"]

# ============================================================================
# MAIN INTERACTIVE LOOP
# ============================================================================

def main():
    """
    Main interactive loop for the agentic system.

    Features:
    - Continuous conversation loop
    - Command system (state, history, exit)
    - File upload support (.txt, .pdf)
    - Graceful error handling
    """

    # Welcome message with feature overview
    print("\n" + "="*70)
    print("🤖 AGENTIC RESUME ANALYZER - FULLY AUTONOMOUS VERSION")
    print("="*70)
    print("Framework-free AI Agent with TRUE Agentic Behavior")
    print("\n✨ AGENTIC FEATURES:")
    print("  ✓ Goal Decomposition & Multi-Step Planning")
    print("  ✓ Self-Reflection & Autonomous Correction")
    print("  ✓ Structured Decision Making (JSON-based)")
    print("  ✓ Confidence-Based Reasoning")
    print("  ✓ Autonomous Multi-Tool Execution")
    print("\n📚 CORE FEATURES:")
    print("  ✓ Agent State Management")
    print("  ✓ ATS Scoring with Explanation")
    print("  ✓ Automated Report Generation")
    print("  ✓ Memory & Context Tracking")
    print("  ✓ Retrieval-Augmented Matching")
    if PDF_SUPPORT:
        print("  ✓ PDF/TXT File Upload Support")
    print("\n💬 COMMANDS:")
    print("  • Type 'exit' to quit")
    print("  • Type 'state' to view agent state")
    print("  • Type 'history' to view execution history")
    print("  • Type 'upload resume.pdf' to load file")
    print("  • Say 'fully evaluate my resume' for autonomous multi-step analysis")
    print("\n" + "="*70)

    # Initialize agent memory
    memory = AgentMemory()

    # Main conversation loop
    while True:
        try:
            user_input = input("\n👤 You: ").strip()

            # Handle exit command
            if user_input.lower() == 'exit':
                print("\n👋 Goodbye! Agent shutting down...\n")
                break

            # Handle state inspection command
            if user_input.lower() == 'state':
                print("\n📊 Current Agent State:")
                for key, value in agent_state.items():
                    if key != "execution_history":  # Don't print full history here
                        print(f"  {key}: {value}")
                continue

            # Handle history command
            if user_input.lower() == 'history':
                print("\n📜 Execution History:")
                for execution in agent_state.get("execution_history", []):
                    print(f"  [{execution['timestamp']}] {execution['tool']}: {execution['result']}")
                continue

            # Handle file upload
            if user_input.lower().startswith('upload '):
                filepath = user_input[7:].strip()
                success, content = load_resume_file(filepath)

                if success:
                    memory.store_resume(content)
                    print(f"✅ File loaded successfully: {filepath}")
                    print("📄 You can now analyze, score, or evaluate this resume.")
                else:
                    print(content)  # Error message
                continue

            # Skip empty input
            if not user_input:
                continue

            # EXECUTE AGENT WITH FULL AUTONOMY
            response = agent_execute(user_input, memory)

            # Store conversation in memory
            memory.store_message("user", user_input)
            memory.store_message("assistant", response)

            # Display agent's response
            print(f"\n🤖 Assistant:\n{response}")

        except KeyboardInterrupt:
            print("\n\n👋 Goodbye! Agent shutting down...\n")
            break
        except Exception as e:
            print(f"\n❌ Error: {str(e)}")
            # Uncomment for debugging:
            # import traceback
            # traceback.print_exc()
            continue

# ============================================================================
# ENTRY POINT
# ============================================================================

if __name__ == "__main__":
    """
    Program entry point.

    To run:
    1. Install dependencies: pip install transformers torch PyPDF2
    2. Run: python agentic_resume_analyzer.py
    3. Or in Colab: Copy entire code and run cell
    """
    main()

# ============================================================================
# END OF CODE
# ============================================================================

🔧 Loading LLM model (this may take 30-60 seconds)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


🤖 AGENTIC RESUME ANALYZER - FULLY AUTONOMOUS VERSION
Framework-free AI Agent with TRUE Agentic Behavior

✨ AGENTIC FEATURES:
  ✓ Goal Decomposition & Multi-Step Planning
  ✓ Self-Reflection & Autonomous Correction
  ✓ Structured Decision Making (JSON-based)
  ✓ Confidence-Based Reasoning
  ✓ Autonomous Multi-Tool Execution

📚 CORE FEATURES:
  ✓ Agent State Management
  ✓ ATS Scoring with Explanation
  ✓ Automated Report Generation
  ✓ Memory & Context Tracking
  ✓ Retrieval-Augmented Matching
  ✓ PDF/TXT File Upload Support

💬 COMMANDS:
  • Type 'exit' to quit
  • Type 'state' to view agent state
  • Type 'history' to view execution history
  • Type 'upload resume.pdf' to load file
  • Say 'fully evaluate my resume' for autonomous multi-step analysis


👤 You: I have 5 years of experience in Python, machine learning, SQL, pandas, TensorFlow, and data visualization. I have built predictive models, worked with large datasets, performed feature engineering, and deployed ML models. I have 